In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
words = ("big", "Data", "Is", "SUPER", "Interesting", "BIG", "data", "IS", "A", "Trending", "technology") 

In [4]:
words_rdd = spark.sparkContext.parallelize(words)

In [5]:
words_rdd.getNumPartitions()

2

In [6]:
# To check default parallelism when we parallelize the data

spark.sparkContext.defaultParallelism

2

In [7]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")

In [8]:
orders_rdd.getNumPartitions()

2

In [10]:
spark.sparkContext.defaultMinPartitions

2

In [3]:
base_rdd = spark.sparkContext.textFile("/public/trendytech/bigLog.txt")

In [4]:
base_rdd.take(10)

['ERROR: Thu Jun 04 10:37:51 BST 2015',
 'WARN: Sun Nov 06 10:37:51 GMT 2016',
 'WARN: Mon Aug 29 10:37:51 BST 2016',
 'ERROR: Thu Dec 10 10:37:51 GMT 2015',
 'ERROR: Fri Dec 26 10:37:51 GMT 2014',
 'ERROR: Thu Feb 02 10:37:51 GMT 2017',
 'WARN: Fri Oct 17 10:37:51 BST 2014',
 'ERROR: Wed Jul 01 10:37:51 BST 2015',
 'WARN: Thu Jul 27 10:37:51 BST 2017',
 'WARN: Thu Oct 19 10:37:51 BST 2017']

In [5]:
base_rdd.getNumPartitions()

3

In [ ]:
# Count the no. of orders under each order status

In [ ]:
# Mathod 1 

In [3]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")

In [5]:
mapped_rdd = orders_rdd.map(lambda x : (x.split(",")[3],1))

In [6]:
mapped_rdd.take(15)

[('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('COMPLETE', 1),
 ('CLOSED', 1),
 ('COMPLETE', 1),
 ('COMPLETE', 1),
 ('COMPLETE', 1),
 ('PROCESSING', 1),
 ('PENDING_PAYMENT', 1),
 ('PENDING_PAYMENT', 1),
 ('PAYMENT_REVIEW', 1),
 ('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('PROCESSING', 1),
 ('COMPLETE', 1)]

In [7]:
results = mapped_rdd.reduceByKey(lambda x,y : x+y)

In [8]:
results.collect()

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899),
 ('PROCESSING', 8275),
 ('PAYMENT_REVIEW', 729),
 ('PENDING', 7610),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558)]

In [ ]:
# Method 2:

In [9]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")

In [10]:
mapped_rdd = orders_rdd.map(lambda x : x.split(",")[3])

In [11]:
mapped_rdd.take(10)

['CLOSED',
 'PENDING_PAYMENT',
 'COMPLETE',
 'CLOSED',
 'COMPLETE',
 'COMPLETE',
 'COMPLETE',
 'PROCESSING',
 'PENDING_PAYMENT',
 'PENDING_PAYMENT']

In [12]:
mapped_rdd.countByValue()

defaultdict(int,
            {'CLOSED': 7556,
             'PENDING_PAYMENT': 15030,
             'COMPLETE': 22899,
             'PROCESSING': 8275,
             'PAYMENT_REVIEW': 729,
             'PENDING': 7610,
             'ON_HOLD': 3798,
             'CANCELED': 1428,
             'SUSPECTED_FRAUD': 1558})

In [ ]:
# map + reduceByKey:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")
mapped_rdd = orders_rdd.map(lambda x : (x.split(",")[3],1))
results = mapped_rdd.reduceByKey(lambda x,y : x+y)
results.collect()

In [ ]:
# countByValue():
orders_rdd = spark.sparkContext.textFile("/public/trendytech/retail_db/orders/part-00000")
mapped_rdd = orders_rdd.map(lambda x : x.split(",")[3])
mapped_rdd.countByValue()